In [1]:
#import yt_dlp
import os
import numpy as np
import librosa
import pandas as pd

In [2]:
# --- Paso 0: Configuración y Funciones Auxiliares ---
import gc

def extract_audio_features(file_path, song_id_txt, sr=None, hop_length=512):
    """
    Extrae características de un archivo de audio, sincronizadas con el beat.

    Args:
        file_path: Ruta al archivo de audio.
        song_id_txt: El ID de la canción (nombre del archivo .txt).
        sr: Frecuencia de muestreo (opcional).  Si es None, usa la original.
        hop_length:  Tamaño del salto (en samples) para características como MFCC.

    Returns:
        Diccionario con características sincronizadas con el beat y songId,
        o None si hay error.
    """
    try:
        y, sr = librosa.load(file_path, sr=sr, duration=120.0)

        # Separar componentes armónicos y percusivos
        y_harmonic, y_percussive = librosa.effects.hpss(y)

        # Seguimiento del beat (en la señal percusiva)
        tempo, beat_frames = librosa.beat.beat_track(y=y_percussive, sr=sr)

        # --- MFCCs ---
        mfcc = librosa.feature.mfcc(y=y, sr=sr, hop_length=hop_length, n_mfcc=13)
        mfcc_delta = librosa.feature.delta(mfcc)
        # Sincronizar con el beat (media)
        beat_mfcc_delta = librosa.util.sync(np.vstack([mfcc, mfcc_delta]), beat_frames)

        # --- Cromagrama ---
        chromagram = librosa.feature.chroma_cqt(y=y_harmonic, sr=sr, hop_length=hop_length)
        # Sincronizar con el beat (mediana)
        beat_chroma = librosa.util.sync(chromagram, beat_frames, aggregate=np.median)

        # --- Combinar Características ---
        beat_features = np.vstack([beat_chroma, beat_mfcc_delta])

        # Convertir a lista y crear diccionario
        features = {
            'song_id': song_id_txt.replace(".txt", ""),
            'beat_features': beat_features.T.tolist()  # Transponer para tener (beats, features)
            # Puedes añadir más características aquí si quieres,
            #  pero DEBEN estar sincronizadas con el beat.
        }
        return features

    finally:
        # Limpieza
        del y, sr, y_harmonic, y_percussive, mfcc, mfcc_delta
        del beat_mfcc_delta, chromagram, beat_chroma, beat_features
        gc.collect()

In [ ]:
import os
import yt_dlp

folder = "Canciones2"
os.makedirs(folder, exist_ok=True)

PO_TOKEN = "MlO6fT_1PrFAPrpBEN2LA0d4vPt_We2gjf4TdmOFjd-49oyE3aF45hsf6iafDUygEBrWdwYee1ZFKcNzaE17fUQEdyR6CUCKV29UHCI6mBjeBFykPA=="

def download_audio(search_query, output_filename):
    ydl_opts = {
        'format': 'bestaudio/best',
        'cookiefile': 'cookiesY.txt',
        'outtmpl': os.path.join(folder, output_filename + '.wav'),
        'noplaylist': True,
        'default_search': 'ytsearch',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
        }],
        'extractor_args': {
            'youtube': f'player_client=web;po_token=web.gvs+{PO_TOKEN}'
        },
        'ignore_no_formats_error': True,
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([search_query])
        print(f"✅ Descargado: {output_filename}.wav")
    except yt_dlp.utils.DownloadError as de:
        print(f"❌ DownloadError al descargar '{search_query}': {de}")
    except Exception as e:
        print(f"❌ Error inesperado al descargar '{search_query}': {e}")

# Ejemplo de uso:
# download_audio("Cold As Ice Atlas Sound", "cold-as-ice")


In [3]:
df = pd.read_csv("Data_completa_enrriquecida.csv")

In [4]:
df.head(5)

,Letra,Idioma,lyrics_state,hasLetra,Lyrics,Lyrics_proces,songId,track_uri,track_name,artist_uri,...,dominance_tags,mbid,spotify_id,genre,cuadrante,cuadReal,lexicones,emociones,emocionesLetra,emocion_mas_comun
0,Data/Letras/1-'Till-I-Collapse-Eminem.txt,en,complete,1.0,TranslationsEspañolTürkçePortuguêsItalianoहिन्...,cause feel tired left left feel weak feel weak...,till-i-collapse-eminem.txt,spotify:track:4xkOaSrkexMciUUogZKVTS,Till I Collapse,spotify:artist:7dGJo4pcD2V6oG8kP0tJRR,...,5.690625,cab93def-26c5-4fb0-bedd-26ec4c1619e1,4xkOaSrkexMciUUogZKVTS,rap,3,2,[0. 2. 0. 0. 0. 2. 6. 0. 0. 2.],"['anticipation', 'negative', 'positive', 'trust']","['3', '2', '1', '3']",3
1,Data/Letras/2-St.-Anger-Metallica.txt,en,complete,1.0,St. Anger Lyrics[Verse]\nSaint Anger 'round my...,saint anger round neck saint anger round neck ...,st.-anger-metallica.txt,spotify:track:3fOc9x06lKJBhz435mInlH,St. Anger,spotify:artist:2ye2Wgw4gimLv2eAKyk1NB,...,5.427250,727a2529-7ee8-4860-aef6-7959884895cb,3fOc9x06lKJBhz435mInlH,metal,3,2,[0. 6. 0. 0. 0. 6. 0. 0. 0. 0.],"['anticipation', 'negative']","['3', '2']",3
2,Data/Letras/3-Speedin'-Rick-Ross.txt,en,complete,1.0,Speedin’ Lyrics[Intro: Rick Ross]\nLegendary\n...,legendary runners know trilla dollar count acc...,speedin-rick-ross.txt,spotify:track:3Y96xd4Ce0J47dcalLrEC8,Speedin',spotify:artist:1sBkRIssrMs1AbVkOJbc7a,...,5.490000,NaN,3Y96xd4Ce0J47dcalLrEC8,rap,3,2,[0. 2. 0. 0. 0. 2. 0. 0. 0. 0.],"['anticipation', 'negative']","['3', '2']",3
3,Data/Letras/4-Bamboo-Banga-M.I.A..txt,en,complete,1.0,"Bamboo Banga Lyrics[Intro]\nRoad runner, road ...",road runner road runner going miles hour road ...,bamboo-banga-m.i.a..txt,spotify:track:6tqFC1DIOphJkCwrjVzPmg,Bamboo Banga,spotify:artist:0QJIPDAEDILuo8AIq3pMuU,...,5.691357,99dd2c8c-e7c1-413e-8ea4-4497a00ffa18,6tqFC1DIOphJkCwrjVzPmg,hip-hop,3,1,[0. 0. 0. 0. 0. 0. 4. 0. 0. 0.],['positive'],['1'],1
4,Data/Letras/5-Die-MF-Die-Dope.txt,en,complete,1.0,Die MF Die Lyrics[Intro]\nDie!\n\n[Verse 1]\nI...,die need forgiveness need hate need acceptance...,die-mf-die-dope.txt,spotify:track:5bU4KX47KqtDKKaLM4QCzh,Die MF Die,spotify:artist:7fWgqc4HJi3pcHhK8hKg2p,...,5.441765,b9eb3484-5e0e-4690-ab5a-ca91937032a5,5bU4KX47KqtDKKaLM4QCzh,metal,3,1,[0. 1. 0. 0. 0. 1. 0. 0. 0. 0.],"['anticipation', 'negative']","['3', '2']",3


In [5]:
df.shape

(24966, 50)

In [6]:
def single_or_mode(series):
    """
    Si la columna tiene exactamente un único valor en el grupo, lo devuelve.
    De lo contrario, devuelve la moda (el valor más frecuente).
    En caso de no existir valor definido, devuelve None.
    """
    sin_nan = series.dropna()
    
    # Valores únicos
    unique_vals = sin_nan.unique()
    
    if len(unique_vals) == 1:
        # Si hay un único valor, lo devolvemos
        return unique_vals[0]
    else:
        modos = sin_nan.mode()
        if len(modos) > 0:
            return modos.iloc[0]
        else:
           
            return None


In [7]:
# Función para manejar duplicados de songId
def consolidate_group(group):
    return pd.Series({
        'Idioma': single_or_mode(group['Idioma']),
        'lyrics_state': single_or_mode(group['lyrics_state']),
        'track_name':single_or_mode(group['track_name']),
        'artist_uri':single_or_mode(group['artist_uri']),
        'hasLetra': group['hasLetra'].max(),
        'Lyrics': max(group['Lyrics'], key=len),
        'Lyrics_proces': max(group['Lyrics_proces'], key=len),
        'track_uri': single_or_mode(group['track_uri']),
        'valence': single_or_mode(group['valence']),
        'valence_tags': single_or_mode(group['valence_tags']),
        'arousal_tags': single_or_mode(group['arousal_tags']),
        'emocion_mas_comun': single_or_mode(group['emocion_mas_comun']),
        'cuadrante': single_or_mode(group['cuadrante']),
        'cuadReal': single_or_mode(group['cuadReal'])
    })



df = df.groupby('songId').apply(consolidate_group).reset_index()
print("Número de songIds únicos después de consolidar:", df['songId'].nunique())

Número de songIds únicos después de consolidar: 24026


C:\Users\andre\AppData\Local\Temp\ipykernel_11552\1057465179.py:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('songId').apply(consolidate_group).reset_index()


In [8]:
song_id_counts = df['songId'].value_counts()

duplicated_song_ids = song_id_counts[song_id_counts > 1].index

df[df['songId'].isin(duplicated_song_ids)].head(20)

,songId,Idioma,lyrics_state,track_name,artist_uri,hasLetra,Lyrics,Lyrics_proces,track_uri,valence,valence_tags,arousal_tags,emocion_mas_comun,cuadrante,cuadReal


In [9]:
import os
import pandas as pd


features_df = pd.read_csv("features_extracted_original.csv")
features_df_2 = pd.read_csv("features_extracted.csv")
features_df_3 = pd.read_csv("features_extracted_final.csv")
features_df_4 = pd.read_csv("features_extracted_final_second.csv")
features_df_5 = pd.read_csv("features_extracted_final_third.csv")
df_audio_6 = pd.read_csv("features_extracted_lastTrue.csv")
df_audio_7 = pd.read_csv("features_extracted_lastTrue2.csv")
df_audio_8 = pd.read_csv("features_extracted_newQ4.csv")
df_audio_9 = pd.read_csv("features_extracted_newQ4_v2.csv")


# Concatenar los DataFrames
concatenated_df = pd.concat([features_df, features_df_2, features_df_3, features_df_4, features_df_5,df_audio_6,df_audio_7,df_audio_8,df_audio_9], ignore_index=True)

# Convertir la columna 'song_id' en un conjunto para búsquedas rápidas
existing_files = set(concatenated_df['song_id'].astype(str) + ".wav")

# Crear una columna 'audio_file' en df con el nombre esperado de archivo
df['audio_file'] = df['songId'].str.replace(".txt", "", regex=False) + ".wav"

# Filtrar para quedarnos solo con las que *no* estén en el conjunto de canciones ya existentes
df_not_in_folder = df[~df['audio_file'].isin(existing_files)].copy()

print("Total de canciones que NO están en Canciones2:", len(df_not_in_folder))

# Seleccionar 200 canciones del cuadranteReal = 3
df_q3 = df_not_in_folder[df_not_in_folder['cuadReal'] == 3].sample(n=350, random_state=42)

# Unir ambos subsets
df_subset = pd.concat([df_q3], ignore_index=True)
print("Canciones seleccionadas para descargar:", len(df_subset))

Total de canciones que NO están en Canciones2: 18477
Canciones seleccionadas para descargar: 350


In [13]:
counts = df_not_in_folder['cuadReal'].value_counts()
print(counts)

cuadReal
2    6336
1    6279
3    5588
4     274
Name: count, dtype: int64


In [10]:
df_subset.head(5)


,songId,Idioma,lyrics_state,track_name,artist_uri,hasLetra,Lyrics,Lyrics_proces,track_uri,valence,valence_tags,arousal_tags,emocion_mas_comun,cuadrante,cuadReal,audio_file
0,silent-night-william-basinski.txt,en,complete,Silent Night,spotify:artist:6u5axd0rpDsWSmzhFfb2VB,1.0,"2 ContributorsArabian Nights,Vol. 2 (Chap. 3) ...",tale king omar bin alnuuman sons sharrkan zau ...,spotify:track:4VHd51AzXeeXlXdh50uruT,0.0266,0.488353,-0.042798,2,0,3,silent-night-william-basinski.wav
1,nada-es-para-siempre-elefante.txt,es,complete,Nada Es para Siempre,spotify:artist:5oYHL2SijkMY52gKIhYJNb,1.0,"Nada Es Para Siempre Lyrics21 de mayo, 3 de la...",mayo tarde toma quiero fotografias para recuer...,spotify:track:2JhxFRDeqtEkpI8a8Rkbq6,0.2920,-0.414321,-1.203860,3,4,3,nada-es-para-siempre-elefante.wav
2,a-slow-song-joe-jackson.txt,en,complete,A Slow Song,spotify:artist:6KOqPxwfNAmZPkiCnDE9yT,1.0,4 ContributorsA Slow Song LyricsMusic has char...,music charms peoples hands savage beast contro...,spotify:track:0nPiPDJapiB1lPyzc8oNML,0.1910,0.578367,-0.216091,3,0,3,a-slow-song-joe-jackson.wav
3,all-this-useless-beauty-june-tabor.txt,en,complete,All This Useless Beauty,spotify:artist:1y2rqr1xpuz2ia93MQ9eEC,1.0,2 ContributorsAll This Useless Beauty Lyrics[V...,times shed tempted spit ladylike imagines live...,spotify:track:6PsGFWEF8AU4vT7WgUyJhi,0.3210,1.130326,1.204911,3,0,3,all-this-useless-beauty-june-tabor.wav
4,betty-kate-walsh.txt,en,complete,Betty,spotify:artist:2SUlbSybJvhWQuzjgaI6FJ,1.0,1 ContributorBetty LyricsBetty knows about her...,betty knows man goes pure gold sent heart open...,spotify:track:1mQ6ekPGRw4kGGPjLgvjEW,0.1510,-0.337688,-0.937632,3,3,3,betty-kate-walsh.wav


In [33]:
df_subset["track_uri"].to_csv("uris.txt", index=False, header=False)

In [11]:
import subprocess

OUTPUT_FOLDER = "Canciones2"

for i, row in df_subset.iterrows():
    uri    = row["track_uri"]
    songid = row["songId"].replace(".txt", "")
    print(f"[{i+1}/{len(df_subset)}] Descargando {songid!r}…")

    try:
        # No usamos check=True aquí, capturamos siempre la salida
        cp = subprocess.run(
            [
                "spotdl", "download", uri,
                "--format", "wav",
                "--output", f"{OUTPUT_FOLDER}/{songid}.{{output-ext}}"
            ],
            capture_output=True, text=True
        )

        if cp.returncode == 0:
            print(f"✅ {songid}.wav descargado correctamente")
        else:
            print(f"❌ Error descargando {songid}:")
            print(cp.stderr or cp.stdout)   # a veces sale por stdout
    except subprocess.CalledProcessError as e:
        # Si decides usar check=True de nuevo, atrápalo así:
        print(f"❌ CalledProcessError en {songid}:")
        print(e.stderr or e.output)
    except Exception as e:
        print(f"❌ Excepción inesperada en {songid}: {e}")

print("🏁 ¡Proceso completado!")

[1/350] Descargando 'silent-night-william-basinski'…
✅ silent-night-william-basinski.wav descargado correctamente
[2/350] Descargando 'nada-es-para-siempre-elefante'…
✅ nada-es-para-siempre-elefante.wav descargado correctamente
[3/350] Descargando 'a-slow-song-joe-jackson'…
✅ a-slow-song-joe-jackson.wav descargado correctamente
[4/350] Descargando 'all-this-useless-beauty-june-tabor'…
✅ all-this-useless-beauty-june-tabor.wav descargado correctamente
[5/350] Descargando 'betty-kate-walsh'…
✅ betty-kate-walsh.wav descargado correctamente
[6/350] Descargando 'cloud-9-rahsaan-patterson'…
✅ cloud-9-rahsaan-patterson.wav descargado correctamente
[7/350] Descargando 'fashionably-lonely-viva-voce'…
✅ fashionably-lonely-viva-voce.wav descargado correctamente
[8/350] Descargando 'troll-lullaby-jarboe'…
✅ troll-lullaby-jarboe.wav descargado correctamente
[9/350] Descargando 'snälla-bli-min-veronica-maggio'…
✅ snälla-bli-min-veronica-maggio.wav descargado correctamente
[10/350] Descargando 'as-tea

In [ ]:
for index, row in df_subset.iterrows():
    # Construir la consulta de búsqueda (puedes ajustar la query para mejorar la precisión)
    query = f"{row['track_name']} {row['artist_uri']}"
    # Remover la extensión ".txt" para el nombre del archivo de audio
    base_filename = row['songId'].replace(".txt", "")
    print(f"Descargando '{query}' como {base_filename}.wav")
    download_audio(query, base_filename)

In [ ]:
del df

In [ ]:
import os
import csv

with open("features_extracted_newQ4_v3.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    # Cabecera (columnas)
    writer.writerow(["song_id", "beat_features"])

    # Recorre solo las canciones que existen en Canciones2
    for index, row in df_subset.iterrows():
        audio_path = os.path.join("Canciones2", row['audio_file'])
        if not os.path.isfile(audio_path):
            print(f"⚠️ Archivo no encontrado, se omite: {audio_path}")
            continue

        print(f"Extrayendo características de {audio_path}")
        features = extract_audio_features(audio_path, row['songId'])

        if features:
            writer.writerow([features["song_id"], features["beat_features"]])

        # Libera memoria de la variable 'features'
        del features

Extrayendo características de Canciones2\silent-night-william-basinski.wav
Extrayendo características de Canciones2\nada-es-para-siempre-elefante.wav
Extrayendo características de Canciones2\a-slow-song-joe-jackson.wav
Extrayendo características de Canciones2\all-this-useless-beauty-june-tabor.wav
Extrayendo características de Canciones2\betty-kate-walsh.wav
Extrayendo características de Canciones2\cloud-9-rahsaan-patterson.wav
Extrayendo características de Canciones2\fashionably-lonely-viva-voce.wav
Extrayendo características de Canciones2\troll-lullaby-jarboe.wav
Extrayendo características de Canciones2\snälla-bli-min-veronica-maggio.wav
Extrayendo características de Canciones2\as-tears-clear-our-eyes-rebekka-bakken.wav
Extrayendo características de Canciones2\roses-indelicates.wav
Extrayendo características de Canciones2\looks-like-im-up-shit-creek-again-tom-waits.wav
Extrayendo características de Canciones2\pastorale-william-coulter.wav
Extrayendo características de Canciones2\tigh